In [1]:
import pandas as pd
import glob
import os
import numpy as np

In [2]:
path1 = r"C:\Users\jacque.trahan\Downloads\Northern Division 2021 Debris Removal Tracker*"
file_type1 = '.xlsx'
load1 = glob.glob(path1 + file_type1)

df1 = pd.read_excel(max(load1, key=os.path.getctime))

In [3]:
path2 = r"C:\Users\jacque.trahan\Downloads\Site Assessment Tracker*"
file_type2 = '.csv'
load2 = glob.glob(path2 + file_type2)

df2 = pd.read_csv(max(load2, key=os.path.getctime))

In [4]:
# df1 = pd.read_excel('../Northern  Dixie fire/Northern Branch trackers/Northern Division 2021 Debris Removal Tracker.xlsx', parse_dates=True)
# df2 = pd.read_csv("../tetra forms ASB and SA/Site Assessment Tracker.csv", parse_dates=True)

In [5]:
df3 =pd.read_excel("../tetra forms ASB and SA/SA_TetraForms1.xlsx", parse_dates=True)

In [6]:
smartsheets = df1.copy()
sa = df2.copy()
tetarforms = df3.copy()

# Site Assessment site up

In [7]:
# sa column selection
sa = sa[["APN", "Survey Date", "Number of Chimney(s)?", 
        "Number of Automobile(s)?", "Number of Motorcycle(s)?", "Number of ATV/UTV/Snowmobiles?", 
       "Number of Ag/Commercial/Construction Equipment?", "Number of Recreational Vehicle(s)?", 
       "Number of Trailer(s)?", "Number of Vessel(s)?", "County"]]

In [8]:
# only use central division APNs and parcel information
sa = sa[sa['County'].isin(['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY'])]

### Get the value counts of APNS in Sa

In [9]:
pd.set_option('display.max_rows',1000)

In [10]:
sa['APN'].value_counts().to_excel('SA power BI APN counts northern.xlsx')

In [11]:
sa['Number of Chimney(s)?'] = np.where(sa['Number of Chimney(s)?'] > 0, 'Yes', 'No')

In [12]:
# set up date columns
sa['Survey Date'] = pd.to_datetime(sa['Survey Date']).dt.date

In [13]:
sa.rename(columns={'APN': 'APN_SA','Survey Date': 'SA Survey Date','Number of Chimney(s)?': 'Chimney',
                   "Number of Automobile(s)?":'Automobiles', 'Number of Motorcycle(s)?':'Motorcycle_SA',
                   "Number of ATV/UTV/Snowmobiles?": 'ATV/UTV/Snowmobiles',
                   "Number of Ag/Commercial/Construction Equipment?":'Ag/Commercial/Construction',
                  "Number of Recreational Vehicle(s)?": 'RV_SA', 'Number of Trailer(s)?':'Trailers_SA',
                   'Number of Vessel(s)?': 'Vessel_SA'}, inplace=True)

In [14]:
# set up numerical values for al vehicles
sa['Automobiles'] = pd.to_numeric(sa['Automobiles'], errors='coerce')
sa['Motorcycle_SA'] = pd.to_numeric(sa['Motorcycle_SA'], errors='coerce')
sa['ATV/UTV/Snowmobiles'] = pd.to_numeric(sa['ATV/UTV/Snowmobiles'], errors='coerce')
sa['Ag/Commercial/Construction'] = pd.to_numeric(sa['Ag/Commercial/Construction'], errors='coerce')
sa['RV_SA'] = pd.to_numeric(sa['RV_SA'], errors='coerce')
sa['Trailers_SA'] = pd.to_numeric(sa['Trailers_SA'], errors='coerce')
sa['Vessel_SA'] = pd.to_numeric(sa['Vessel_SA'], errors='coerce')

In [15]:
sa.fillna({'SA Survey Date':0,
           'Automobiles': 0,
           'Motorcycle_SA': 0,
           'ATV/UTV/Snowmobiles': 0,
           'Ag/Commercial/Construction': 0,
           'RV_SA': 0,
           'Trailers_SA': 0,
           'Vessel_SA': 0}, inplace=True)

In [16]:
sa.columns = map(str.upper, sa.columns)

# Tetra Forms setup

In [17]:
# get just the counties we need
tetarforms = tetarforms[tetarforms['county'].isin(['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY'])]

### Get the value counts of APNS in Tetra Forms for central

In [18]:
tetarforms['apn'].value_counts().to_excel('Tetra Forms APN counts SA Northern.xlsx')

In [19]:
# format dates 
tetarforms['survey_date'] = pd.to_datetime(tetarforms['survey_date']).dt.date

In [20]:
#get columsn we need
tetarforms = tetarforms[['apn','survey_date','chimney_quantity', 'automobile_quantity', 'motorcycle_quantity', 'atv_utv_quantity',
                        'comm_equip_quantity', 'rvs_quantity', 'trailer_quantity', 'vessel_quantity']]

In [21]:
# tetarforms['motorcycle_quantity'].dtypes

#### add fill NA to all blanks

In [22]:
tetarforms['chimney_quantity'] = np.where(tetarforms['chimney_quantity'] > 0,'Yes', 'No')

In [23]:
tetarforms.fillna({'survey_date':0,
           'automobile_quantity': 0,
           'motorcycle_quantity': 0,
           'atv_utv_quantity': 0,
           'comm_equip_quantity': 0,
           'rvs_quantity': 0,
           'trailer_quantity': 0,
           'vessel_quantity': 0}, inplace=True)

In [24]:
tetarforms.rename(columns={'apn': 'APN_TF', 'motorcycle_quantity':'motorcycle_quantity_TF'}, inplace=True)

In [25]:
tetarforms.columns = map(str.upper, tetarforms.columns)

In [26]:
# merge1 tetraforms and SA
merge1 = sa.merge(tetarforms,
                 left_on='APN_SA',
                 right_on='APN_TF',
                 how='outer',
                 suffixes=('_SA', '_TF'))

# Smart sheet set up

In [27]:
# smart sheet column selection
smartsheets = smartsheets[['APN_ROW Segment','Street #','Street Name', 'Structural Status','County',
                           'Chimney',
                           'Site Assessment',
                          'Number of Passenger Vehicles', 'Number of ATV_UTV_snowmobiles',
                          'Number of Ag_Commercial_Construction Equipment','Number of Motorcycles',
                          'Number of RV\'s', 'Number of Trailers', 'Number of Vessels (Boat_Jetski)', 'Total Number of Vehicles']]

In [28]:
smartsheets = smartsheets.copy()

In [29]:
# take of the time on the dates
smartsheets['Site Assessment'] = smartsheets['Site Assessment'].dt.date

In [30]:
smartsheets.fillna({'Chimney': 'No',
                    'Site Assessment': 0,
                    'Number of Passenger Vehicles': 0,
                    'Number of ATV_UTV_snowmobiles': 0,
                    'Number of Ag_Commercial_Construction Equipment': 0,
                    'Number of Motorcycles': 0,
                    'Number of RV\'s':0,
                    'Number of Trailers': 0,
                    'Number of Vessels (Boat_Jetski)': 0}, inplace=True)

In [31]:
smartsheets.rename(columns={'County':'County_SS','Site Assessment': 'SA Survey Date', 'Number of Passenger Vehicles': 'Automobiles',
                           'Number of ATV_UTV_snowmobiles': 'ATV/UTV/Snowmobiles',
                           'Number of Ag_Commercial_Construction Equipment': 'Ag/Commercial/Construction',
                           'Number of Motorcycles': 'Motorcycles_SS', 'Number of RV\'s': 'RV_SS',
                           'Number of Trailers':'Trailers_SS','Number of Vessels (Boat_Jetski)':'Vessel_SS'}, inplace=True)

In [32]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [33]:
# merge both data frames for QC
finals = smartsheets.merge(merge1,
                         left_on='APN_ROW SEGMENT',
                         right_on='APN_SA',
                         how='outer',
                         suffixes=('_SS', '_SA'))

In [34]:
# column orinizationa
final = finals[['APN_ROW SEGMENT','APN_SA', 'APN_TF','STREET #','STREET NAME','STRUCTURAL STATUS','COUNTY_SS',
               'SA SURVEY DATE_SS', 'SA SURVEY DATE_SA','SURVEY_DATE',
               'CHIMNEY_SS','CHIMNEY_SA','CHIMNEY_QUANTITY',
               'AUTOMOBILES_SS', 'AUTOMOBILES_SA','AUTOMOBILE_QUANTITY',
               'MOTORCYCLES_SS', 'MOTORCYCLE_SA', 'MOTORCYCLE_QUANTITY_TF',
              'ATV/UTV/SNOWMOBILES_SS', 'ATV/UTV/SNOWMOBILES_SA','ATV_UTV_QUANTITY',
              'AG/COMMERCIAL/CONSTRUCTION_SS', 'AG/COMMERCIAL/CONSTRUCTION_SA','COMM_EQUIP_QUANTITY',
              'RV_SS', 'RV_SA','RVS_QUANTITY',
              'TRAILERS_SS', 'TRAILERS_SA','TRAILER_QUANTITY',
              'VESSEL_SS','VESSEL_SA','VESSEL_QUANTITY',]]

In [35]:
# Fill in the na and blanks with 0's

In [36]:
final.fillna({'SA SURVEY DATE_SA':0,
           'SURVEY_DATE': 0,
           'CHIMNEY_SA': 0,
           'CHIMNEY_QUANTITY': 0,
           'AUTOMOBILES_SA': 0,
           'AUTOMOBILE_QUANTITY': 0,
           'MOTORCYCLE_SA': 0,
           'MOTORCYCLE_QUANTITY_TF': 0,
             'ATV/UTV/SNOWMOBILES_SA': 0,
             'ATV_UTV_QUANTITY': 0,
             'AG/COMMERCIAL/CONSTRUCTION_SA': 0,
             'COMM_EQUIP_QUANTITY': 0,
             'RV_SA': 0,
             'RVS_QUANTITY': 0,
             'TRAILERS_SA':0,
             'TRAILER_QUANTITY': 0,
             'VESSEL_SA': 0,
             'VESSEL_QUANTITY': 0}, inplace=True)

C:\Users\jacque.trahan\AppData\Roaming\Python\Python39\site-packages\pandas\core\series.py:4463: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().fillna(


# compare function for each category

In [37]:
def compare_cols(df,comp1,comp2,comp3):
    if (df[comp1] == df[comp2]) & (df[comp1] == df[comp3]):
        return 1
    else:
        return 0

# Survey date checks

In [38]:
final.insert(final.columns.get_loc('SURVEY_DATE')+1, "Survey Matches",
            final.apply(compare_cols, comp1='SA SURVEY DATE_SS', comp2='SA SURVEY DATE_SA', comp3='SURVEY_DATE', axis=1))

# CHIMNEY

In [39]:
final.insert(final.columns.get_loc('CHIMNEY_QUANTITY')+1, "Chimney Macthes",
            final.apply(compare_cols, comp1='CHIMNEY_SS', comp2='CHIMNEY_SA', comp3='CHIMNEY_QUANTITY', axis=1))

# AUTOMOBILES

In [40]:
final.insert(final.columns.get_loc('AUTOMOBILE_QUANTITY')+1, "AUTOMOBILE-M",
            final.apply(compare_cols, comp1='AUTOMOBILES_SS', comp2='AUTOMOBILES_SA', comp3='AUTOMOBILE_QUANTITY', axis=1))

# MOTORCYCLES

In [41]:
final.insert(final.columns.get_loc('MOTORCYCLE_QUANTITY_TF')+1, "Motorcycle-M",
            final.apply(compare_cols, comp1='MOTORCYCLES_SS', comp2='MOTORCYCLE_SA', comp3='MOTORCYCLE_QUANTITY_TF', axis=1))

# ATV/UTV/SNOWMOBILES

In [42]:
final.insert(final.columns.get_loc('ATV_UTV_QUANTITY')+1, "ATV-M",
            final.apply(compare_cols,
                        comp1='ATV/UTV/SNOWMOBILES_SS', comp2='ATV/UTV/SNOWMOBILES_SA', comp3='ATV_UTV_QUANTITY', axis=1))

# AG/COMMERCIAL/CONSTRUCTION

In [43]:
final.insert(final.columns.get_loc('COMM_EQUIP_QUANTITY')+1, "COMM-M",
            final.apply(compare_cols,
                        comp1='AG/COMMERCIAL/CONSTRUCTION_SS',
                        comp2='AG/COMMERCIAL/CONSTRUCTION_SA', comp3='COMM_EQUIP_QUANTITY', axis=1))

# RV

In [44]:
final.insert(final.columns.get_loc('RVS_QUANTITY')+1, "RV-M",
            final.apply(compare_cols, comp1='RV_SS', comp2='RV_SA', comp3='RVS_QUANTITY', axis=1))

# TRAILERS

In [45]:
final.insert(final.columns.get_loc('TRAILER_QUANTITY')+1, "Trailer-M",
            final.apply(compare_cols, comp1='TRAILERS_SS', comp2='TRAILERS_SA', comp3='TRAILER_QUANTITY', axis=1))

# VESSEL

In [46]:
final.insert(final.columns.get_loc('VESSEL_QUANTITY')+1, "Vessel-M",
            final.apply(compare_cols, comp1='VESSEL_SS', comp2='VESSEL_SA', comp3='VESSEL_QUANTITY', axis=1))

# color in the compare columns

In [47]:
final_style = final.style.set_properties(**{'background-color': 'lightblue'},
                          subset=['Survey Matches', 'Chimney Macthes','AUTOMOBILE-M','Motorcycle-M',"ATV-M",
                                  'COMM-M',"RV-M",'Trailer-M' ,'Vessel-M'])

In [48]:
final_style.to_excel('Northern Dixie Fire SA Audit.xlsx')